In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder\
.master("spark://192.168.2.70:7077") \
.appName("ludvigwesterholm_firstpart")\
.config("spark.dynamicAllocation.enabled", True)\
.config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
.config("spark.shuffle.service.enabled", True)\
.config("spark.dynamicAllocation.executorIdleTimeout","30s")\
.config("spark.cores.max", 4)\
.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/20 13:07:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def add1(a, b):
    # associative and commutative!
    return a + b

rdd = spark_session.sparkContext.parallelize(range(10**7))

result = rdd.filter(lambda x: x % 2 == 0)\
    .map(lambda x: x ** 2)\
    .reduce(add1)

print(result)

166666616666670000000


In [3]:
from operator import add
sc = spark_session.sparkContext

#reading the text files
swetext = sc.textFile("hdfs://192.168.2.70:9000/europarl/europarl-v7.sv-en.sv")
engtext = sc.textFile("hdfs://192.168.2.70:9000/europarl/europarl-v7.sv-en.en")

print(f"Number of Swedish lines: {swetext.count()}")
print(f"Number of English lines: {engtext.count()}")

Number of Swedish lines: 1862234


Number of English lines: 1862234


In [4]:
print(f"Number of Swedish partitions: {swetext.getNumPartitions()}")
print(f"Number of English partitions: {engtext.getNumPartitions()}")

Number of Swedish partitions: 3
Number of English partitions: 2


In [8]:
#Turns letters into lowercase
def rdd_lower(rdd):
    return rdd.map(lambda x: x.lower())

#splits a string into 2 depending on a given string
def rdd_split(rdd, split_token):
    return rdd.map(lambda x: x.split(split_token))

#turning to lowercase
swetext_lower = rdd_lower(swetext)
#splits string on space
swetext_split = rdd_split(swetext_lower, " ")

engtext_lower = rdd_lower(engtext)
engtext_split = rdd_split(engtext_lower, " ")

#comparing that the text is the same for the preprocessed vs original
print(swetext_split.take(10))
print(swetext.take(10))


print(engtext_split.take(10))
print(engtext.take(10))

#making sure that the preprocessing didn't change the number of lines
swetext_split.count()
engtext_split.count()


[['återupptagande', 'av', 'sessionen'], ['jag', 'förklarar', 'europaparlamentets', 'session', 'återupptagen', 'efter', 'avbrottet', 'den', '17', 'december.', 'jag', 'vill', 'på', 'nytt', 'önska', 'er', 'ett', 'gott', 'nytt', 'år', 'och', 'jag', 'hoppas', 'att', 'ni', 'haft', 'en', 'trevlig', 'semester.'], ['som', 'ni', 'kunnat', 'konstatera', 'ägde', '"den', 'stora', 'år', '2000-buggen"', 'aldrig', 'rum.', 'däremot', 'har', 'invånarna', 'i', 'ett', 'antal', 'av', 'våra', 'medlemsländer', 'drabbats', 'av', 'naturkatastrofer', 'som', 'verkligen', 'varit', 'förskräckliga.'], ['ni', 'har', 'begärt', 'en', 'debatt', 'i', 'ämnet', 'under', 'sammanträdesperiodens', 'kommande', 'dagar.'], ['till', 'dess', 'vill', 'jag', 'att', 'vi,', 'som', 'ett', 'antal', 'kolleger', 'begärt,', 'håller', 'en', 'tyst', 'minut', 'för', 'offren', 'för', 'bl.a.', 'stormarna', 'i', 'de', 'länder', 'i', 'europeiska', 'unionen', 'som', 'drabbats.'], ['jag', 'ber', 'er', 'resa', 'er', 'för', 'en', 'tyst', 'minut.'], 

['Återupptagande av sessionen', 'Jag förklarar Europaparlamentets session återupptagen efter avbrottet den 17 december. Jag vill på nytt önska er ett gott nytt år och jag hoppas att ni haft en trevlig semester.', 'Som ni kunnat konstatera ägde "den stora år 2000-buggen" aldrig rum. Däremot har invånarna i ett antal av våra medlemsländer drabbats av naturkatastrofer som verkligen varit förskräckliga.', 'Ni har begärt en debatt i ämnet under sammanträdesperiodens kommande dagar.', 'Till dess vill jag att vi, som ett antal kolleger begärt, håller en tyst minut för offren för bl.a. stormarna i de länder i Europeiska unionen som drabbats.', 'Jag ber er resa er för en tyst minut.', '(Parlamentet höll en tyst minut.)', 'Fru talman! Det gäller en ordningsfråga.', 'Ni känner till från media att det skett en rad bombexplosioner och mord i Sri Lanka.', 'En av de personer som mycket nyligen mördades i Sri Lanka var Kumar Ponnambalam, som besökte Europaparlamentet för bara några månader sedan.']


[['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,', 'on', 'behalf', 'of', '

['Resumption of the session', 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.', "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.", 'You have requested a debate on this subject in the course of the next few days, during this part-session.', "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.", "Please rise, then, for this minute' s silence.", "(The House rose and observed a minute' s silence)", 'Madam President, on a point of order.', 'You will be aware from the press and television that there have been a num

1862234

In [48]:
#Question 1.A.3

from operator import add

#turn everything into a single array of words
engtext_flat = engtext_split.flatMap(lambda x: x)

#turns each word into a tuple
engtext_tuple = engtext_flat.map(lambda x: (x,1))

#summs the value for each key into a unique tuple using the add function from operator
engtext_words = engtext_tuple.reduceByKey(add)

#sorts the tuples in descending size, meaning biggest first going down, and printing the first 10, 
#ie the 10 most common words
engtext_words.sortBy(lambda x: x[1], ascending = False).take(10)



#"the, of, to, and, in, that, a, is, for and we" are among the most common words used in the
#english language, which would make this result seem plausible.


[('the', 3498375),
 ('of', 1659758),
 ('to', 1539760),
 ('and', 1288401),
 ('in', 1085993),
 ('that', 797516),
 ('a', 773522),
 ('is', 758050),
 ('for', 534242),
 ('we', 522849)]